# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

print(da.columns)
# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI","BPXDI1",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')


## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [9]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

0.031

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [11]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [12]:
model = sm.GEE.from_formula("BPXDI1 ~ BMXBMI+RIAGENDRx+RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

0.030, it decreased slightly

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [21]:
da_male = da[da['RIAGENDRx']=='Male'].reset_index(drop=True)
da_female=da[da['RIAGENDRx']=="Female"].reset_index(drop=True)

In [22]:
da_male.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,BPXDI1,SMQ020,SDMVSTRA,SDMVPSU,group,RIAGENDRx
0,128.0,62,1,3,5.0,27.8,70.0,1,125,1,1251,Male
1,146.0,53,1,3,3.0,30.8,88.0,1,125,1,1251,Male
2,138.0,78,1,3,3.0,28.8,46.0,1,131,1,1311,Male
3,110.0,22,1,4,4.0,28.0,70.0,1,128,2,1282,Male
4,178.0,56,1,4,3.0,33.6,116.0,2,126,2,1262,Male


In [23]:
da_female.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,BPXDI1,SMQ020,SDMVSTRA,SDMVPSU,group,RIAGENDRx
0,132.0,56,2,3,5.0,42.4,72.0,2,131,1,1311,Female
1,100.0,42,2,4,4.0,20.3,70.0,2,126,2,1262,Female
2,116.0,72,2,1,2.0,28.6,58.0,2,128,1,1281,Female
3,120.0,32,2,1,4.0,28.2,70.0,2,125,1,1251,Female
4,104.0,30,2,2,4.0,26.6,50.0,1,124,1,1241,Female


In [24]:
model_male = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_male)
result = model_male.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.035


In [25]:
model_female = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da_female)
result = model_female.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.029


__Q3a.__ What do you learn by comparing these two fitted models?

the correlation between males in clusters is more as compared to female

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [28]:
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

In [29]:
da.dropna(inplace=True)

In [30]:
# Fit a multilevel (mixed effects) model to handle dependent data
model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(DMDEDUC2x)",
           groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                Mixed Linear Model Regression Results
======================================================================
Model:                 MixedLM     Dependent Variable:     BPXDI1     
No. Observations:      5100        Method:                 REML       
No. Groups:            30          Scale:                  154.2352   
Min. group size:       105         Likelihood:             -20111.9561
Max. group size:       226         Converged:              Yes        
Mean group size:       170.0                                          
----------------------------------------------------------------------
                            Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                   65.646    1.032 63.631 0.000 63.624 67.668
RIAGENDRx[T.Male]            2.755    0.351  7.855 0.000  2.068  3.443
C(DMDEDUC2x)[T.HS]          -1.093    0.521 -2.098 0.036 -2.115 -0.072
C(DMDEDUC2x)[T.SomeCollege] -0.428    0.484 -0.883 0.377 -1.377  0.521
C(DMDEDUC2x)[T.lt9]         -0.955    0.641 -1.488 0.137 -2.212  0.303
C(DMDEDUC2x)[T.x9_11]       -0.219    0.630 -0.347 0.729 -1.454  1.016
RIDAGEYR                    -0.039    0.010 -3.871 0.000 -0.059 -0.019
BMXBMI                       0.186    0.026  7.283 0.000  0.136  0.236
group Var                    4.170    0.108                           
======================================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

pretty strong, 2*gender effect(almost), 4.170/0.186 times bmi effect

In [31]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(DMDEDUC2x)",
           groups="group", vc_formula={"age_cen": "1"}, data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Mixed Linear Model Regression Results
=========================================================================
Model:                  MixedLM      Dependent Variable:      BPXSY1     
No. Observations:       5100         Method:                  REML       
No. Groups:             30           Scale:                   256.9676   
Min. group size:        105          Likelihood:              -21414.4041
Max. group size:        226          Converged:               Yes        
Mean group size:        170.0                                            
-------------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-------------------------------------------------------------------------
Intercept                   112.720    1.211 93.053 0.000 110.346 115.094
RIAGENDRx[T.Male]             3.509    0.453  7.749 0.000   2.621   4.396
C(DMDEDUC2x)[T.HS]            3.014    0.673  4.478 0.000   1.695   4.333
C(DMDEDUC2x)[T.SomeCollege]   2.006    0.626  3.207 0.001   0.780   3.232
C(DMDEDUC2x)[T.lt9]           4.241    0.828  5.119 0.000   2.618   5.865
C(DMDEDUC2x)[T.x9_11]         4.206    0.814  5.167 0.000   2.611   5.802
age_cen                       0.456    0.013 34.708 0.000   0.430   0.482
BMXBMI                        0.296    0.033  8.969 0.000   0.231   0.361
age_cen Var                   8.755    0.169                             
=========================================================================

"""

__Q4b:__ Include a random intercept for age, and describe your findings.